In [6]:
london = 'https://www.earthcam.com/world/england/london/abbeyroad/?cam=abbeyroad_uk'
timesquare = 'https://www.earthcam.com/usa/newyork/timessquare/?cam=tsrobo1'
dublin = 'https://www.earthcam.com/world/ireland/dublin/?cam=templebar'

In [9]:
import os
import time
from datetime import timedelta
import numpy as np

import cv2
import streamlink
from imageai.Detection import VideoObjectDetection, ObjectDetection
# from apscheduler.schedulers.blocking import BlockingScheduler
# from selenium import webdriver


class CountingObject(object):
    """
    A class of counting objects
    """
    
    algos = {"resnet": "resnet50_coco_best_v2.0.1.h5", "yolov3": "yolo.h5", "yolo_tiny": "yolo-tiny.h5"}
    
    def __init__(self, stream_link):
        self.stream_link = stream_link
        self.streams = streamlink.streams(stream_link)
        if self.streams is None:
            raise ValueError("cannot open the stream link %s" % stream_link)
        
        #change 1, self detect quality
        q = list(self.streams.keys())[0]
        self.stream = self.streams['%s'%q]
        
        self.target_img_path = os.getcwd()
        
        self.detector = ObjectDetection()
        if self.detector is None:
            raise ValueError("Detector of objects is None")
        
        
    def detector_init(self, algo="resnet", speed="nomal"):
        """
        Must be invoked after instantiate for initialize a object detector. 
        
        Args:
            algo (str): The algorithm of object detection tasks. "resnet"(default), "yolov3", "yolo_tiny".
            speed (str): The detection speed for object detetion tasks. "normal"(default), "fast", "faster" , "fastest" and "flash".
        
        Returns:
            void
        
        """
        print("The current algorithm is %s. "%algo)
        print("The current speed is %s. "%speed)
        if algo == "resnet":
            self.detector.setModelTypeAsRetinaNet()
            self.detector.setModelPath(os.path.join(self.target_img_path, self.algos["resnet"]))
           
        elif algo == "yolov3":
            self.detector.setModelTypeAsYOLOv3()
            self.detector.setModelPath(os.path.join(self.target_img_path, self.algos["yolov3"]))
            
        elif algo == "yolo_tiny":
            self.detector.setModelTypeAsTinyYOLOv3()
            self.detector.setModelPath(os.path.join(self.target_img_path, self.algos["yolo_tiny"]))
           
        else:
            print("Given algorithm of object detection is invalid.")
            return
        
        self.detector.loadModel(detection_speed=speed)
        self.custom_objects = self.detector.CustomObjects(person=True)

    def put_text_to_img(self, img, text,pos = (50,50),fontColor=(0,0,255),lineType=2):
        if img is None:
            print("Put text to a none image.")
            return
        
        font                  = cv2.FONT_HERSHEY_SIMPLEX
        fontScale             = 1

        cv2.putText(img, text,  
                    pos, 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
    #change 2 add nim,tin
    def capture_frame_by_stream(self,imagename="stream",mpp=30,num_im=6,time_interval=10):
        print("The current conuting function is based on capture frame by stream.")
        print("The current minimum percent probability: %d" %mpp)
        print("The current required number of imgaes: %d" %num_im)
        print("The current required time interval: %d" %time_interval)
        
        
        number_list = []
              
        for i in range(num_im):
        
            video_cap = cv2.VideoCapture(self.stream.url)
            if video_cap is None:
                print("Open webcam [%s] failed." % self.stream.url)
                return
            else:
                ret, frame = video_cap.read()

                if not ret:
                    print("Captured frame is broken.")
                    return 
                else:
                    print("-----------------------------------------------------")
                    print("Capturing frame %d."%i)
                    detections = self.detector.detectCustomObjectsFromImage(custom_objects=self.custom_objects, 
                          input_type="array", 
                          input_image=frame, 
                          output_image_path=os.path.join(self.target_img_path , "%s_%d_result.jpg" %(imagename,i)), 
                          minimum_percentage_probability=mpp)
                    
                    print("The number of person in frame %d (%s_%d_result.jpg ):"%(i,imagename,i), len(detections))
                    print("The current time in frame %d (%s_%d_result.jpg ):"%(i,imagename,i), time.asctime())
                   
                    
                    img = cv2.imread(os.path.join(self.target_img_path, "%s_%d_result.jpg" %(imagename,i)))
                    # put the number of persons to the image and put timestamp to the image
                    self.put_text_to_img(img, "The number of person:%s "%str(len(detections)))
                    self.put_text_to_img(img, "The current time:%s "%time.asctime(),pos=(50,450),fontColor=())
                    
#                     cv2.imshow("image", img)
                    cv2.imwrite("%s_%d_result.jpg" %(imagename,i),img)
#                     cv2.waitKey(0) # blocked until pressing Enter key
#                     cv2.destroyAllWindows()
                    number_list.append(len(detections))                   
            video_cap.release()
            time.sleep(time_interval)
        print(number_list)
        return number_list



          

    def capture_frame_by_screenshot(self,imagename="screenshot",mpp=30,num_im=6,time_interval=10):
        print("The current conuting function is based on capture frame by screenshot.")
        print("The current minimum percent probability: %d" %mpp)
        print("The current required number of imgaes: %d" %num_im)
        print("The current required time interval: %d" %time_interval)
        number_list = []
        
        
        if self.driver is None:
            print("Web driver is none.")
            return 
        else:
            
            for i in range(num_im):
                print("-----------------------------------------------------")
                print('Taking screenshot %d...'%i)
                self.driver.save_screenshot("%s_%d.jpg" %(imagename,i))
                detections = self.detector.detectCustomObjectsFromImage(custom_objects=self.custom_objects, 
                                                  input_image=os.path.join(self.target_img_path , "%s_%d.jpg" %(imagename,i)), 
                                                  output_image_path=os.path.join(self.target_img_path , "%s_%d.jpg" %(imagename,i)), 
                                                  minimum_percentage_probability=mpp)
                
                print("The number of person in frame %d (%s_%d_result.jpg ):"%(i,imagename,i), len(detections))
                print("The current time in frame %d (%s_%d_result.jpg ):"%(i,imagename,i), time.asctime())
                
                img = cv2.imread(os.path.join(self.target_img_path, "%s_%d.jpg" %(imagename,i)))
                # put the number of persons to the image
                self.put_text_to_img(img, "The number of person is:%s"%str(len(detections)))
                self.put_text_to_img(img, "The current time:%s "%time.asctime(),pos=(50,450),fontColor=())

               
                cv2.imwrite("%s_%d_result.jpg" %(imagename,i),img)
                time.sleep(time_interval)
                number_list.append(len(detections)) 
        
        self.driver.quit()
        print(number_list)
        return number_list

    def init_webdriver(self):
        self.driver = webdriver.Chrome(os.path.join('E:/Googledriver/chromedriver_win32/chromedriver.exe'))  # Optional argument, if not specified will search path.
        self.driver.get(self.stream_link)
        time.sleep(15) # Jump over the ads
        
if __name__ == "__main__":
#     scheduler = BlockingScheduler()
    print("Starting...")
    counting_person = CountingObject(dublin)
    counting_person.detector_init()

    d = counting_person.capture_frame_by_stream(imagename="dublin",num_im=100,time_interval=300)

#     counting_person.init_webdriver()
#     counting_person.capture_frame_by_screenshot(num_im=2)

        
    print('###exit.')
        
#  raises errors, nor ready for using.
#     scheduler.add_job(capture_frame, 'interval', seconds=5, args=[cap, detect, custom_objects, target_img_path])
#     scheduler.start()



Starting...
The current algorithm is resnet. 
The current speed is nomal. 
The current conuting function is based on capture frame by stream.
-----------------------------------------------------
Capturing frame 0.
The number of person in frame 0 (dublin_0_result.jpg ): 12
The current time in frame 0 (dublin_0_result.jpg ): Sat Dec  1 11:26:47 2018
-----------------------------------------------------
Capturing frame 1.
The number of person in frame 1 (dublin_1_result.jpg ): 15
The current time in frame 1 (dublin_1_result.jpg ): Sat Dec  1 11:31:54 2018
-----------------------------------------------------
Capturing frame 2.
The number of person in frame 2 (dublin_2_result.jpg ): 9
The current time in frame 2 (dublin_2_result.jpg ): Sat Dec  1 11:37:01 2018
-----------------------------------------------------
Capturing frame 3.
The number of person in frame 3 (dublin_3_result.jpg ): 17
The current time in frame 3 (dublin_3_result.jpg ): Sat Dec  1 11:42:07 2018
-----------------------

-----------------------------------------------------
Capturing frame 38.
The number of person in frame 38 (dublin_38_result.jpg ): 16
The current time in frame 38 (dublin_38_result.jpg ): Sat Dec  1 14:42:46 2018
-----------------------------------------------------
Capturing frame 39.
The number of person in frame 39 (dublin_39_result.jpg ): 22
The current time in frame 39 (dublin_39_result.jpg ): Sat Dec  1 14:47:53 2018
-----------------------------------------------------
Capturing frame 40.
The number of person in frame 40 (dublin_40_result.jpg ): 22
The current time in frame 40 (dublin_40_result.jpg ): Sat Dec  1 14:53:00 2018
-----------------------------------------------------
Capturing frame 41.
The number of person in frame 41 (dublin_41_result.jpg ): 18
The current time in frame 41 (dublin_41_result.jpg ): Sat Dec  1 14:58:06 2018
-----------------------------------------------------
Capturing frame 42.
The number of person in frame 42 (dublin_42_result.jpg ): 21
The curre

The number of person in frame 76 (dublin_76_result.jpg ): 10
The current time in frame 76 (dublin_76_result.jpg ): Sat Dec  1 17:56:54 2018
-----------------------------------------------------
Capturing frame 77.
The number of person in frame 77 (dublin_77_result.jpg ): 15
The current time in frame 77 (dublin_77_result.jpg ): Sat Dec  1 18:02:01 2018
-----------------------------------------------------
Capturing frame 78.
The number of person in frame 78 (dublin_78_result.jpg ): 23
The current time in frame 78 (dublin_78_result.jpg ): Sat Dec  1 18:07:07 2018
-----------------------------------------------------
Capturing frame 79.
The number of person in frame 79 (dublin_79_result.jpg ): 20
The current time in frame 79 (dublin_79_result.jpg ): Sat Dec  1 18:12:14 2018
-----------------------------------------------------
Capturing frame 80.
The number of person in frame 80 (dublin_80_result.jpg ): 14
The current time in frame 80 (dublin_80_result.jpg ): Sat Dec  1 18:17:21 2018
----

In [10]:
print (d)

[12, 15, 9, 17, 12, 14, 5, 14, 12, 19, 7, 10, 19, 12, 19, 6, 21, 20, 18, 26, 24, 21, 15, 19, 29, 23, 19, 28, 19, 24, 17, 22, 16, 26, 25, 23, 15, 18, 16, 22, 22, 18, 21, 25, 15, 21, 18, 15, 19, 17, 26, 18, 25, 19, 10, 21, 19, 18, 24, 15, 22, 19, 35, 19, 21, 24, 23, 28, 20, 10, 19, 17, 18, 21, 12, 25, 10, 15, 23, 20, 14, 14, 22, 21, 19, 16, 18, 18, 22, 23, 18, 14, 19, 22, 17, 18, 28, 24, 20, 18]


In [11]:
np.mean(d)

18.8